In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
import sys
import math
import csv

from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split 
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import Ridge
# the new model
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten


from sklearn import metrics
from sklearn.metrics import classification_report 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
# Function importing Dataset 
def importdata(trainFile, testFile): 
    train = pd.read_csv(trainFile, sep= ',', header = None) 
    test = pd.read_csv(testFile, sep= ',', header = None) 
    return train, test    

# Function to split target from data 
def splitdataset(train, test): 
    ohe = OneHotEncoder(sparse=True)
    objInTrain = len(train)

    allData = pd.concat([train, test], ignore_index=True, sort =False, axis=0)
    AllDataWihoutClass = allData.iloc[:, :-1]
    AllDataWihoutClassOnlyNominals = AllDataWihoutClass.select_dtypes(include=['object'])
    AllDataWihoutClassNoNominals = AllDataWihoutClass.select_dtypes(exclude=['object'])

    encAllDataWihoutClassNominals = ohe.fit_transform(AllDataWihoutClassOnlyNominals)
    encAllDataWihoutClassNominalsToPanda = pd.DataFrame(encAllDataWihoutClassNominals.toarray())
    
    if AllDataWihoutClassOnlyNominals.shape[1] > 0:
      codAllDataAgain = pd.concat([encAllDataWihoutClassNominalsToPanda, AllDataWihoutClassNoNominals], ignore_index=True, sort =False, axis=1)
    else:
      codAllDataAgain = AllDataWihoutClass

    # Seperating the target variable 
    X_train = codAllDataAgain[:objInTrain]
    y_train = train.values[:, -1]

    X_test = codAllDataAgain[objInTrain:]
    y_test = test.values[:, -1]
    
    mm_scaler = MinMaxScaler()
    X_train_minmax = pd.DataFrame(mm_scaler.fit_transform(X_train[X_train.columns]), index=X_train.index, columns=X_train.columns)
    X_test_minmax = pd.DataFrame(mm_scaler.transform(X_test[X_test.columns]), index=X_test.index, columns=X_test.columns)
    
    std_scaler = StandardScaler()
    X_train_std = pd.DataFrame(std_scaler.fit_transform(X_train[X_train.columns]), index=X_train.index, columns=X_train.columns)
    X_test_std = pd.DataFrame(std_scaler.transform(X_test[X_test.columns]), index=X_test.index, columns=X_test.columns)
    
    X_train_minmax_std = pd.DataFrame(std_scaler.fit_transform(X_train_minmax[X_train_minmax.columns]), index=X_train_minmax.index, columns=X_train_minmax.columns)
    X_test_minmax_std = pd.DataFrame(std_scaler.transform(X_test_minmax[X_test_minmax.columns]), index=X_test_minmax.index, columns=X_test_minmax.columns)
    
    return X_train, X_test, y_train, y_test, X_train_minmax, X_test_minmax, X_train_std, X_test_std, X_train_minmax_std, X_test_minmax_std

# Function to make predictions 
def prediction(X_test, clf_object):  
    y_pred = clf_object.score_samples(X_test) 
    return y_pred 

def result_of_Class(y_test, y_pred, saveFile):       
    np.savetxt(saveFile, y_pred, fmt='%.4f')

In [ ]:
rootDir = '/content/drive/My Drive/Colab Notebooks/OCC_FromTavo'

# Uncommment for using DNN
clf_classif = DNN2()
data = {'folder_name': [],
                'auc': []}
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Directorio encontrado: %s' % dirName)
    print("************************************ DIRECTORIO **************************************")
    if len(fileList) > 0: 
        arr_auc = []
        arr_folder_name = dirName.split("/")
        folder_name = arr_folder_name[len(arr_folder_name) - 1]
        completed_name = folder_name + "-5-"
        for i in range(1, int(len(fileList) / 2) + 1):
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!") 
            trainFile = str(dirName) + '/' + completed_name + str(i) +"tra.dat.csv"
            testFile = str(dirName) + '/' + completed_name + str(i) +"tst.dat.csv"
            print('trainFile ' + trainFile)
            print('testFile ' + testFile)
            # Loading data 
            train, test = importdata(trainFile, testFile)

            # Training
            X_train, X_test, y_train, y_test, X_train_minmax, X_test_minmax, X_train_std, X_test_std, X_train_minmax_std, X_test_minmax_std = splitdataset(train, test)

            # Performing training 
            clf_classif.fit(X_train, y_train) 

            # Operational Phase 
            y_pred_classif = prediction(X_test, clf_classif) 

            auc = metrics.roc_auc_score(y_test,  y_pred_classif)
            arr_auc.append(1 - auc if auc < 0.5 else auc)
            print("AUC: "+str(1 - auc if auc < 0.5 else auc))
            print(y_pred_classif)

        print('AUC!! ' + str(arr_auc))
        aver_auc = sum(arr_auc) / len(arr_auc)
        print('aver_auc!! ' + str(aver_auc))
        data['folder_name'].append(folder_name)
        data['auc'].append(aver_auc)
        print('data[auc] ' + str(data['auc']))

    df = pd.DataFrame(data, columns = ['folder_name', 'auc'])    
    df.to_csv('occ_results.csv')